In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
import glasbey

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

s_palette = sns.cubehelix_palette(as_cmap=True)
g_palette = glasbey.create_palette()
blue_palette = sns.cubehelix_palette(5, rot=-.25, light=.7)

In [3]:
from src.utils.tracklets import import_tracklets

roots = ["embryo018"]
datapath = Path().cwd().parent / "data" / "interim" / "confocal"
plotpath = datapath / "plots"
plotpath.mkdir(exist_ok=True)

spots, tracklets, metadata, tracklets_joined = import_tracklets(datapath, roots)

In [41]:
print(spots[roots[0]].columns)
print(tracklets[roots[0]].columns)

Index(['Unnamed: 0', 'ID', 'track_id', 'tracklet_id', 'distance_from_edge',
       'parent_id', 'daughter_id', 'roi', 'FRAME', 'POSITION_X', 'POSITION_Y',
       'POSITION_Z', 'ELLIPSE_MAJOR', 'ELLIPSE_MINOR', 'ELLIPSE_THETA',
       'ELLIPSE_Y0', 'ELLIPSE_X0', 'ELLIPSE_ASPECTRATIO', 'CIRCULARITY',
       'AREA', 'SHAPE_INDEX', 'MEDIAN_INTENSITY_CH1', 'time', 'um_from_edge',
       'um_x', 'um_y', 'ap_position', 'track_n_tracklets', 'cycle',
       'edge_position'],
      dtype='object')
Index(['start_time', 'end_time', 'start_frame', 'end_frame', 'length',
       'source_spot', 'sink_spot', 'mean_ap_position', 'source_ap_position',
       'sink_ap_position', 'initial_x', 'initial_y', 'final_x', 'final_y',
       'initial_x_um', 'initial_y_um', 'final_x_um', 'final_y_um', 'track_id',
       'mean_edge_distance', 'track_n_tracklets', 'cycle', 'embryo',
       'tracklet_id', 'parent_tracklet', 'n_children', 'e_id', 'e_parent_id'],
      dtype='object')


In [5]:
root = "embryo018"
raw_img = tifffile.imread(datapath / root / f"{root}_MaxIP_bgs.tif")
first_frame = 81
last_Frame = 167

In [54]:
from cv2 import polylines, LINE_AA

def make_outline_image(spots_df, raw, frame):
    img = raw[frame]
    img = np.stack([img, img, img], axis=-1).astype(np.uint8)
    spots_frame = spots_df[spots_df["FRAME"] == frame]
    for i, spot in spots_frame.iterrows():
        roi = spot["roi"]
        # convert string to list of floats
        roi = [float(x) for x in roi[1:-1].split(", ")]
        roi_x = roi[::2]
        roi_y = roi[1::2]
        x_c, y_x = spot["POSITION_X"], spot["POSITION_Y"]
        roi_x = [x + x_c for x in roi_x]
        roi_x.append(roi_x[0])
        roi_y = [y + y_x for y in roi_y]
        roi_y.append(roi_y[0])
        roi = np.reshape(np.round(np.stack([roi_x, roi_y], -1)), (1, -1, 2)).astype(np.int32)
        polylines(img, roi, True, spot["cycle_12_color"], 1, lineType=LINE_AA)
    return img

def assign_cycle_12_color(spots_df):
    spots_df["cycle_12_id"] = 0
    spots_df["cycle_12_id"] = (spots_df["cycle"] == 11) * spots_df["tracklet_id"]
    spots_df["cycle_12_id"] = spots_df["cycle_12_id"] + (spots_df["cycle"] == 12) * spots_df["parent_tracklet_id"]
    
    g_palette = glasbey.create_palette(palette_size = len(spots_df["cycle_12_id"].unique()), as_hex=False)
    color_dict = dict(zip(spots_df["cycle_12_id"].unique(), g_palette))
    color_dict = {k: tuple(round(vi*255) for vi in v) for k, v in color_dict.items()}
    spots_df["cycle_12_color"] = spots_df["cycle_12_id"].map(color_dict)
    
    return spots_df
    
    
spots_df = spots[root].copy()
tracklet = tracklets[root].copy()
spots_df["parent_tracklet_id"] = spots_df["tracklet_id"].map(tracklet.set_index("tracklet_id")["parent_tracklet"])

spots_df = assign_cycle_12_color(spots_df)

# outlined = make_outline_image(spots_df, raw_img, 181)
# 
# plt.figure(figsize=(10, 10))
# plt.imshow(outlined)
# plt.savefig(plotpath / "nucleus_outline.png", dpi=300)

out_arr = []
for i in tqdm(range(first_frame, last_Frame)):
    out_arr.append(make_outline_image(spots_df, raw_img, i))
    
tifffile.imwrite(datapath / root / f"{root}_nucleus_outline.tif", np.stack(out_arr))

100%|██████████| 86/86 [00:03<00:00, 27.84it/s]
